<a href="https://colab.research.google.com/github/adrianacupp/fake_new_classification/blob/main/text_title_joined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing

In [1]:
import pandas as pd
import nltk
#import spacy
from nltk import ngrams
from nltk.tokenize import word_tokenize 
from sklearn.datasets import make_classification
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
import re
import pycld2 as cld2
from langdetect import detect
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score, classification_report 
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.metrics import RocCurveDisplay, plot_roc_curve

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/odelia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 10.2 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=901235e49d43ae9742504fe83c40a2e3bf39cf7e76a4435cfc019730747af159
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [2]:
df = pd.read_csv("data/WELFake_Dataset.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)
df

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
...,...,...,...
72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   71576 non-null  object
 1   text    72095 non-null  object
 2   label   72134 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [4]:
df['label'].value_counts()

1    37106
0    35028
Name: label, dtype: int64

In [5]:
df[df['title'].isna() & df['text'].isna()]

,title,text,label


In [6]:
df[df['title'].isna() | df['text'].isna()]

,title,text,label
1,NaN,Did they post their votes for Hillary already?,1
43,NaN,True. Hillary needs a distraction and what bet...,1
162,NaN,All eyes on Electoral delegates. The People kn...,1
185,NaN,Cool,1
269,NaN,A leading US senator: US Supporting War in Syr...,1
...,...,...,...
71484,NaN,Another Arab supremacist masturbation fantasy....,1
71521,NaN,I'm sure they drastically changed accounting m...,1
71540,NaN,It's easy to imagine Obama or Kerry pissing hi...,1
71570,NaN,Ever since the powers to be assassinated JFK A...,1


In [7]:
df['text'] = df['title'] + ' ' + df['text']

In [8]:
df.drop(['title'], axis=1,inplace=True)

In [9]:
df

,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,NaN,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",0
4,SATAN 2: Russia unvelis an image of its terrif...,1
...,...,...
72129,Russians steal research on Trump in hack of U....,0
72130,WATCH: Giuliani Demands That Democrats Apolog...,1
72131,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,Trump tussle gives unpopular Mexican leader mu...,0


In [10]:
df.dropna(axis=0, how='any', inplace=True)

In [11]:
df.reset_index(drop=True, inplace=True)
df

,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0
3,SATAN 2: Russia unvelis an image of its terrif...,1
4,About Time! Christian Group Sues Amazon and SP...,1
...,...,...
71532,Russians steal research on Trump in hack of U....,0
71533,WATCH: Giuliani Demands That Democrats Apolog...,1
71534,Migrants Refuse To Leave Train At Refugee Camp...,0
71535,Trump tussle gives unpopular Mexican leader mu...,0


# Text preprocessing
## With NLTK

### Cleaning text

In [16]:
def clean(text):
    text = str(text).lower()
    text = re.sub(r'\s+', ' ', text)           # sequences of white spaces
    text = re.sub(r'[^\x00-\x7F]+',' ', text)  # Removing all the non ASCII characters
    text = re.sub(r'\s+',' ', text)            # Replacing multiple Spaces with Single Space
    text = re.sub(r'\.{2,}', ' ', text)        # Replacing Two or more dots with one
    text = re.sub(r'[^\x00-\x7F]+',' ', text)  # Removing all the non ASCII characters
    text = re.sub(r'\W+',' ', text)            # Replace everything non-alpahnumeric with a space
    return text.strip()

In [17]:
df['text_clean'] = df['text'].map(clean)

In [18]:
df["empty_cell_text"] = df['text_clean'].str.contains(r'^\s*$', na=False)

In [19]:
df

,text,label,text_clean,empty_cell_text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,law enforcement on high alert following threat...,False
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,unbelievable obama s attorney general says mos...,False
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0,bobby jindal raised hindu uses story of christ...,False
3,SATAN 2: Russia unvelis an image of its terrif...,1,satan 2 russia unvelis an image of its terrify...,False
4,About Time! Christian Group Sues Amazon and SP...,1,about time christian group sues amazon and spl...,False
...,...,...,...,...
71532,Russians steal research on Trump in hack of U....,0,russians steal research on trump in hack of u ...,False
71533,WATCH: Giuliani Demands That Democrats Apolog...,1,watch giuliani demands that democrats apologiz...,False
71534,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...,False
71535,Trump tussle gives unpopular Mexican leader mu...,0,trump tussle gives unpopular mexican leader mu...,False


### Removing empty cells

In [20]:
df.drop(df.loc[df["empty_cell_text"]].index, inplace=True)
df

,text,label,text_clean,empty_cell_text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,law enforcement on high alert following threat...,False
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,unbelievable obama s attorney general says mos...,False
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0,bobby jindal raised hindu uses story of christ...,False
3,SATAN 2: Russia unvelis an image of its terrif...,1,satan 2 russia unvelis an image of its terrify...,False
4,About Time! Christian Group Sues Amazon and SP...,1,about time christian group sues amazon and spl...,False
...,...,...,...,...
71532,Russians steal research on Trump in hack of U....,0,russians steal research on trump in hack of u ...,False
71533,WATCH: Giuliani Demands That Democrats Apolog...,1,watch giuliani demands that democrats apologiz...,False
71534,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...,False
71535,Trump tussle gives unpopular Mexican leader mu...,0,trump tussle gives unpopular mexican leader mu...,False


In [21]:
df.reset_index(drop=True, inplace=True)
df.drop(columns=["empty_cell_text"], inplace=True)

In [22]:
df

,text,label,text_clean
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,law enforcement on high alert following threat...
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,unbelievable obama s attorney general says mos...
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0,bobby jindal raised hindu uses story of christ...
3,SATAN 2: Russia unvelis an image of its terrif...,1,satan 2 russia unvelis an image of its terrify...
4,About Time! Christian Group Sues Amazon and SP...,1,about time christian group sues amazon and spl...
...,...,...,...
71520,Russians steal research on Trump in hack of U....,0,russians steal research on trump in hack of u ...
71521,WATCH: Giuliani Demands That Democrats Apolog...,1,watch giuliani demands that democrats apologiz...
71522,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...
71523,Trump tussle gives unpopular Mexican leader mu...,0,trump tussle gives unpopular mexican leader mu...


### Language detection

In [23]:
def detect_lang(text):
    _, _, _, detected_language = cld2.detect(text, returnVectors=True)
    return str(detected_language)

In [24]:
df['text_lang'] = df['text_clean'].map(detect_lang)

In [25]:
df['text_lang'].astype(str)

0        ((0, 4965, 'ENGLISH', 'en'),)
1         ((0, 343, 'ENGLISH', 'en'),)
2        ((0, 7896, 'ENGLISH', 'en'),)
3        ((0, 1970, 'ENGLISH', 'en'),)
4        ((0, 1561, 'ENGLISH', 'en'),)
                     ...              
71520    ((0, 4727, 'ENGLISH', 'en'),)
71521    ((0, 3673, 'ENGLISH', 'en'),)
71522    ((0, 2836, 'ENGLISH', 'en'),)
71523    ((0, 3356, 'ENGLISH', 'en'),)
71524    ((0, 2681, 'ENGLISH', 'en'),)
Name: text_lang, Length: 71525, dtype: object

In [26]:
df['text_lang'] = ~df["text_lang"].str.contains('ENGLISH|Unknown', regex=True)

In [27]:
df

,text,label,text_clean,text_lang
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,law enforcement on high alert following threat...,False
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,unbelievable obama s attorney general says mos...,False
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0,bobby jindal raised hindu uses story of christ...,False
3,SATAN 2: Russia unvelis an image of its terrif...,1,satan 2 russia unvelis an image of its terrify...,False
4,About Time! Christian Group Sues Amazon and SP...,1,about time christian group sues amazon and spl...,False
...,...,...,...,...
71520,Russians steal research on Trump in hack of U....,0,russians steal research on trump in hack of u ...,False
71521,WATCH: Giuliani Demands That Democrats Apolog...,1,watch giuliani demands that democrats apologiz...,False
71522,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...,False
71523,Trump tussle gives unpopular Mexican leader mu...,0,trump tussle gives unpopular mexican leader mu...,False


In [28]:
df.drop(df.loc[df["text_lang" ]].index, inplace=True)

In [29]:
df

,text,label,text_clean,text_lang
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,law enforcement on high alert following threat...,False
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,unbelievable obama s attorney general says mos...,False
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0,bobby jindal raised hindu uses story of christ...,False
3,SATAN 2: Russia unvelis an image of its terrif...,1,satan 2 russia unvelis an image of its terrify...,False
4,About Time! Christian Group Sues Amazon and SP...,1,about time christian group sues amazon and spl...,False
...,...,...,...,...
71520,Russians steal research on Trump in hack of U....,0,russians steal research on trump in hack of u ...,False
71521,WATCH: Giuliani Demands That Democrats Apolog...,1,watch giuliani demands that democrats apologiz...,False
71522,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...,False
71523,Trump tussle gives unpopular Mexican leader mu...,0,trump tussle gives unpopular mexican leader mu...,False


In [30]:
df.reset_index(drop=True, inplace=True)

In [31]:
df.head(50)

,text,label,text_clean,text_lang
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,law enforcement on high alert following threat...,False
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,unbelievable obama s attorney general says mos...,False
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0,bobby jindal raised hindu uses story of christ...,False
3,SATAN 2: Russia unvelis an image of its terrif...,1,satan 2 russia unvelis an image of its terrify...,False
4,About Time! Christian Group Sues Amazon and SP...,1,about time christian group sues amazon and spl...,False
5,DR BEN CARSON TARGETED BY THE IRS: “I never ha...,1,dr ben carson targeted by the irs i never had ...,False
6,HOUSE INTEL CHAIR On Trump-Russia Fake Story: ...,1,house intel chair on trump russia fake story n...,False
7,Sports Bar Owner Bans NFL Games…Will Show Only...,1,sports bar owner bans nfl games will show only...,False
8,Latest Pipeline Leak Underscores Dangers Of Da...,1,latest pipeline leak underscores dangers of da...,False
9,GOP Senator Just Smacked Down The Most Puncha...,1,gop senator just smacked down the most punchab...,False


In [32]:
df.drop(columns=["text_lang"], inplace=True)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71425 entries, 0 to 71424
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        71425 non-null  object
 1   label       71425 non-null  int64 
 2   text_clean  71425 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


In [34]:
df.to_csv("data/df_pre_tok_text.csv")

### Tokenizing

In [35]:
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")

In [36]:
df['token_text'] = df.apply(lambda row: nltk.word_tokenize(row['text_clean']), axis=1)

In [ ]:
#df.token_text[0]

### POS tagging

In [37]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [ ]:
#df['tag_text'] = df.apply(lambda row: nltk.pos_tag(row['token_text']), axis=1)

### Lemmatizing tagged words

In [38]:
lem = WordNetLemmatizer()

In [39]:
def lemmatize(words):
    lemmatized_words = [lem.lemmatize(word) for word in words]
    return lemmatized_words

In [40]:
df['lem_text'] = df.apply(lambda row: lemmatize(row['token_text']), axis=1)

In [41]:
df

,text,label,text_clean,token_text,lem_text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,law enforcement on high alert following threat...,"[law, enforcement, on, high, alert, following,...","[law, enforcement, on, high, alert, following,..."
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,unbelievable obama s attorney general says mos...,"[unbelievable, obama, s, attorney, general, sa...","[unbelievable, obama, s, attorney, general, sa..."
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0,bobby jindal raised hindu uses story of christ...,"[bobby, jindal, raised, hindu, uses, story, of...","[bobby, jindal, raised, hindu, us, story, of, ..."
3,SATAN 2: Russia unvelis an image of its terrif...,1,satan 2 russia unvelis an image of its terrify...,"[satan, 2, russia, unvelis, an, image, of, its...","[satan, 2, russia, unvelis, an, image, of, it,..."
4,About Time! Christian Group Sues Amazon and SP...,1,about time christian group sues amazon and spl...,"[about, time, christian, group, sues, amazon, ...","[about, time, christian, group, sue, amazon, a..."
...,...,...,...,...,...
71420,Russians steal research on Trump in hack of U....,0,russians steal research on trump in hack of u ...,"[russians, steal, research, on, trump, in, hac...","[russian, steal, research, on, trump, in, hack..."
71421,WATCH: Giuliani Demands That Democrats Apolog...,1,watch giuliani demands that democrats apologiz...,"[watch, giuliani, demands, that, democrats, ap...","[watch, giuliani, demand, that, democrat, apol..."
71422,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...,"[migrants, refuse, to, leave, train, at, refug...","[migrant, refuse, to, leave, train, at, refuge..."
71423,Trump tussle gives unpopular Mexican leader mu...,0,trump tussle gives unpopular mexican leader mu...,"[trump, tussle, gives, unpopular, mexican, lea...","[trump, tussle, give, unpopular, mexican, lead..."


In [ ]:
#df.to_csv("data/df_lemmatized_text.csv")

### Stopwords

In [42]:
stop_words = list(stopwords.words('english')) 
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [43]:
for i in range(len(stop_words)):
    stop_words[i] = re.sub(r"\s*'\s*\w*","",stop_words[i])

In [44]:
df["stop_text"] = df["lem_text"].apply(lambda x: ' '.join([word for word in x if word not in (stop_words)]))

In [45]:
df['stop_text'][0]

'law enforcement high alert following threat cop white 9 11by blacklivesmatter fyf911 terrorist video comment expected barack obama member fyf911 fukyoflag blacklivesmatter movement called lynching hanging white people cop encouraged others radio show tuesday night turn tide kill white people cop send message killing black people america one f yoflag organizer called sunshine ha radio blog show hosted texas called sunshine f ing opinion radio show snapshot fyf911 lolatwhitefear twitter page 9 53 p show wa urging supporter call fyf911 tonight continue dismantle illusion white snapshot twitter radio call invite fyf911the radio show aired 10 00 p eastern standard time show caller clearly call lynching killing white people 2 39 minute clip radio show heard wa provided breitbart texas someone would like referred hannibal ha already received death threat result interrupting fyf911 conference call unidentified black man said mother f kers start f ing like u bunch ni er takin one u roll said c

In [ ]:
#df.to_csv("data/df_stopwords_text.csv")

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


### BOW with countvec [ignore]

In [47]:
def create_vectorizer(sentences):
    vectorizer = CountVectorizer(max_features=100)
    X = vectorizer.fit_transform(sentences)
    return (vectorizer, X)

In [48]:
(vectorizer, X) = create_vectorizer(df.stop_text)

In [49]:
print(vectorizer.get_feature_names())

['2016', 'according', 'also', 'america', 'american', 'attack', 'back', 'bill', 'campaign', 'case', 'city', 'clinton', 'come', 'could', 'country', 'court', 'day', 'democrat', 'donald', 'election', 'even', 'first', 'former', 'get', 'go', 'going', 'government', 'group', 'ha', 'hillary', 'house', 'know', 'last', 'law', 'leader', 'life', 'like', 'made', 'make', 'many', 'may', 'medium', 'member', 'million', 'month', 'mr', 'much', 'national', 'need', 'new', 'news', 'obama', 'official', 'one', 'part', 'party', 'people', 'percent', 'police', 'policy', 'political', 'president', 'presidential', 'public', 'report', 'republican', 'reuters', 'right', 'russia', 'said', 'say', 'security', 'see', 'since', 'state', 'support', 'take', 'thing', 'think', 'time', 'told', 'trump', 'two', 'united', 'video', 'vote', 'wa', 'want', 'war', 'washington', 'way', 'week', 'well', 'white', 'woman', 'work', 'world', 'would', 'year', 'york']


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print(X)

  (0, 33)	1
  (0, 93)	10
  (0, 84)	2
  (0, 51)	1
  (0, 42)	1
  (0, 56)	10
  (0, 3)	1
  (0, 53)	11
  (0, 28)	3
  (0, 86)	5
  (0, 79)	1
  (0, 97)	2
  (0, 36)	6
  (0, 69)	14
  (0, 72)	2
  (0, 74)	1
  (0, 82)	2
  (0, 77)	1
  (0, 88)	2
  (0, 25)	2
  (0, 13)	1
  (0, 48)	3
  (0, 70)	2
  (0, 92)	1
  (0, 90)	1
  :	:
  (71424, 39)	1
  (71424, 76)	1
  (71424, 23)	1
  (71424, 67)	1
  (71424, 80)	2
  (71424, 0)	1
  (71424, 62)	1
  (71424, 98)	1
  (71424, 60)	2
  (71424, 61)	2
  (71424, 22)	1
  (71424, 8)	2
  (71424, 47)	1
  (71424, 12)	1
  (71424, 31)	4
  (71424, 49)	1
  (71424, 14)	1
  (71424, 95)	2
  (71424, 2)	2
  (71424, 63)	2
  (71424, 46)	1
  (71424, 87)	1
  (71424, 11)	12
  (71424, 29)	7
  (71424, 99)	1


In [50]:
denseX = X.todense()

In [51]:
denseX[0]

matrix([[ 0,  0,  0,  1,  2,  1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
          1,  0,  0,  0,  0,  1,  0,  1,  0,  2,  0,  0,  3,  0,  0,  0,
          0,  1,  0,  1,  6,  0,  1,  1,  0,  0,  1,  0,  0,  0,  0,  0,
          3,  0,  1,  1,  0, 11,  0,  0, 10,  0,  1,  0,  0,  0,  0,  0,
          0,  0,  0,  1,  0, 14,  2,  0,  2,  0,  1,  0,  1,  1,  0,  1,
          2,  0,  2,  0,  2,  0,  5,  0,  2,  0,  1,  0,  1, 10,  0,  0,
          0,  2,  0,  0]])

In [52]:
df

,text,label,text_clean,token_text,lem_text,stop_text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,law enforcement on high alert following threat...,"[law, enforcement, on, high, alert, following,...","[law, enforcement, on, high, alert, following,...",law enforcement high alert following threat co...
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,unbelievable obama s attorney general says mos...,"[unbelievable, obama, s, attorney, general, sa...","[unbelievable, obama, s, attorney, general, sa...",unbelievable obama attorney general say charlo...
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0,bobby jindal raised hindu uses story of christ...,"[bobby, jindal, raised, hindu, uses, story, of...","[bobby, jindal, raised, hindu, us, story, of, ...",bobby jindal raised hindu us story christian c...
3,SATAN 2: Russia unvelis an image of its terrif...,1,satan 2 russia unvelis an image of its terrify...,"[satan, 2, russia, unvelis, an, image, of, its...","[satan, 2, russia, unvelis, an, image, of, it,...",satan 2 russia unvelis image terrifying new su...
4,About Time! Christian Group Sues Amazon and SP...,1,about time christian group sues amazon and spl...,"[about, time, christian, group, sues, amazon, ...","[about, time, christian, group, sue, amazon, a...",time christian group sue amazon splc designati...
...,...,...,...,...,...,...
71420,Russians steal research on Trump in hack of U....,0,russians steal research on trump in hack of u ...,"[russians, steal, research, on, trump, in, hac...","[russian, steal, research, on, trump, in, hack...",russian steal research trump hack u democratic...
71421,WATCH: Giuliani Demands That Democrats Apolog...,1,watch giuliani demands that democrats apologiz...,"[watch, giuliani, demands, that, democrats, ap...","[watch, giuliani, demand, that, democrat, apol...",watch giuliani demand democrat apologize trump...
71422,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...,"[migrants, refuse, to, leave, train, at, refug...","[migrant, refuse, to, leave, train, at, refuge...",migrant refuse leave train refugee camp hungar...
71423,Trump tussle gives unpopular Mexican leader mu...,0,trump tussle gives unpopular mexican leader mu...,"[trump, tussle, gives, unpopular, mexican, lea...","[trump, tussle, give, unpopular, mexican, lead...",trump tussle give unpopular mexican leader muc...


## Modeling

### TF-IDF 

In [ ]:
#df = pd.read_csv("data/df_stopwords.csv")
#df.drop(columns=["Unnamed: 0"], inplace=True)
#df

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71425 entries, 0 to 71424
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        71425 non-null  object
 1   label       71425 non-null  int64 
 2   text_clean  71425 non-null  object
 3   token_text  71425 non-null  object
 4   lem_text    71425 non-null  object
 5   stop_text   71425 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.3+ MB


In [54]:
X_train, X_test, Y_train, Y_test = train_test_split(df['stop_text'],
df['label'],
test_size=0.2,
random_state=42,
stratify=df['label'])

In [55]:
print ('Size of Training Data ', X_train.shape[0])
print ('Size of Test Data ', X_test.shape[0])
print ('Distribution of classes in Training Data :')
print ('Fake item ', str(sum(Y_train == 1)/ len(Y_train) * 100.0))
print ('Real item ', str(sum(Y_train == 0)/ len(Y_train) * 100.0))
print ('Distribution of classes in Testing Data :')
print ('Fake item ', str(sum(Y_test == 1)/ len(Y_test) * 100.0))
print ('Real item ', str(sum(Y_test == 0)/ len(Y_test) * 100.0))

Size of Training Data  57140
Size of Test Data  14285
Distribution of classes in Training Data :
Fake item  50.95904795239762
Real item  49.04095204760238
Distribution of classes in Testing Data :
Fake item  50.955547777388865
Real item  49.04445222261113


In [56]:
tfidf = TfidfVectorizer(analyzer = 'word', max_features = 20000, ngram_range=(1,2))
X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [57]:
SVC = LinearSVC(random_state=42, tol=1e-5)
SVC.fit(X_train_tf, Y_train)

LinearSVC(random_state=42, tol=1e-05)

In [58]:
Y_pred = SVC.predict(X_test_tf)
print ('Accuracy Score - ', accuracy_score(Y_test, Y_pred))
print ('ROC-AUC Score - ', roc_auc_score(Y_test, Y_pred))

Accuracy Score -  0.9662583129156458
ROC-AUC Score -  0.9661549790482413


In [59]:
tfidf.get_feature_names()

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '00 pm',
 '000',
 '000 000',
 '000 american',
 '000 email',
 '000 job',
 '000 mile',
 '000 new',
 '000 people',
 '000 per',
 '000 refugee',
 '000 rohingya',
 '000 syrian',
 '000 troop',
 '000 vote',
 '000 year',
 '01',
 '02',
 '03',
 '04',
 '05',
 '07',
 '08',
 '09',
 '10',
 '10 000',
 '10 2016',
 '10 2017',
 '10 billion',
 '10 day',
 '10 million',
 '10 minute',
 '10 month',
 '10 percent',
 '10 point',
 '10 time',
 '10 year',
 '100',
 '100 000',
 '100 billion',
 '100 day',
 '100 million',
 '100 people',
 '100 percent',
 '100 year',
 '1000',
 '101',
 '10th',
 '11',
 '11 16',
 '11 2001',
 '11 2016',
 '11 2017',
 '11 attack',
 '11 day',
 '11 million',
 '11 percent',
 '11 year',
 '110',
 '115',
 '11th',
 '12',
 '12 000',
 '12 2016',
 '12 billion',
 '12 million',
 '12 month',
 '12 percent',
 '12 year',
 '120',
 '120 day',
 '125',
 '12th',
 '13',
 '13 2016',
 '13 2017',
 '13 million',
 '13 percent',
 '13 year',
 '130',
 '135',
 '13th',
 '14',
 '14 2016',
 '14 2017',
 '14 million',
 '